# Arpitha Coorg
#### Question 1: I used generative AI for getting started on  problem 5 to help debug my gradient descent implementation since I did not understand the error I was running into. 

In [89]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [90]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
target = "Sleep_Quality"


In [91]:
#2a)

non_numeric = (
    train
    .drop(columns=[target], errors = "ignore")
    .select_dtypes(exclude="number")
    .columns
    .tolist()
)

cols_drop = non_numeric.copy()

if "ID" in train.columns:
    cols_drop.append("ID")

train_clean = train.drop(columns=cols_drop)
test_clean = test.drop(columns=cols_drop)

print("Removed Columns:" , cols_drop)
print("Train shape:", train_clean.shape)
print("Test shape:", test_clean.shape)

Removed Columns: ['Gender', 'Occupation', 'BMI_Category', 'Blood_Pressure', 'Sleep_Disorder', 'ID']
Train shape: (300, 7)
Test shape: (74, 7)


##### 2a: Removed Columns other than ID: ['Gender', 'Occupation', 'BMI_Category', 'Blood_Pressure', 'Sleep_Disorder']



In [128]:
#2b)
X_train = train_clean.drop(columns=[target])
Y_train = train_clean[target]
X_test = test_clean.drop(columns=[target])
Y_test = test_clean[target]

lr = LinearRegression()
lr.fit(X_train,Y_train)
yhat_train = lr.predict(X_train)
mse_train = mean_squared_error(Y_train,yhat_train)
r2_train = r2_score(Y_train,yhat_train)

coef_df = pd.DataFrame ({
    "feature": X_train.columns,
    "coef": lr.coef_
})
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_df = coef_df.sort_values("abs_coef", ascending=False)


print("Intercept:", lr.intercept_)
print("Training MSE:", mse_train)
print("Training R^2:", r2_train)
print(coef_df)

Intercept: 4.614076015572055
Training MSE: 0.13100264319632646
Training R^2: 0.9064110852787418
          feature      coef  abs_coef
1  Sleep_Duration  0.663838  0.663838
3    Stress_Level -0.322086  0.322086
4      Heart_Rate -0.021018  0.021018
0             Age  0.013840  0.013840
2  Activity_Level -0.000594  0.000594
5     Daily_Steps  0.000092  0.000092


In [129]:
#2c)
yhat_test = lr.predict(X_test)
mse_test = mean_squared_error(Y_test,yhat_test)
r2_test = r2_score(Y_test,yhat_test)
print("Testing MSE:", mse_test)
print("Testing R^2:", r2_test)

Testing MSE: 0.12919025461693848
Testing R^2: 0.9127041171912198


In [130]:
#2d)
cols = ["BMI_Category", "Sleep_Disorder"]

def keep_numeric (df, target, cols):
    df2 = df.copy()
    if "ID" in df2.columns:
        df2 = df2.drop(columns=["ID"])
    numeric_cols = (
        df2.drop(columns=[target], errors = "ignore")
        .select_dtypes(include = "number")
        .columns
        .tolist()
    )
    keep_cols = [target] + numeric_cols + cols
    return df2[keep_cols]
train_second = keep_numeric(train, target, cols)
test_second = keep_numeric(test, target, cols)
enc_train_second = pd.get_dummies(train_second, columns=cols, drop_first=True)
enc_test_second = pd.get_dummies(test_second, columns=cols, drop_first=True)
enc_train_second, enc_test_second = enc_train_second.align(
    enc_test_second, join = "left", axis = 1, fill_value= 0
)

X_train2 = enc_train_second.drop(columns = [target])
Y_train2 = enc_train_second[target]
X_test2 = enc_test_second.drop(columns=[target])
Y_test2 = enc_test_second[target]

lr2 = LinearRegression()
lr2.fit(X_train2, Y_train2)
yhat_train2 = lr2.predict(X_train2)
yhat_test2 = lr2.predict(X_test2)
new_train_mse = mean_squared_error(Y_train2, yhat_train2)
new_train_r2 = r2_score(Y_train2, yhat_train2)
new_test_mse = mean_squared_error(Y_test2, yhat_test2)
new_test_r2 = r2_score(Y_test2, yhat_test2)

print("Training MSE:", new_train_mse)
print("Training R2:", new_train_r2)
print("Testing MSE:", new_test_mse)
print("Testing R2:", new_test_r2)

Training MSE: 0.08934405084462635
Training R2: 0.9361721828557429
Testing MSE: 0.08169486165862917
Testing R2: 0.9447974996985867


##### 2d: I chose BMI_Category and Sleep_Disorder because they have relevant health related information. These features were encoded using one-hot encoding to avoid having any artificial numerical ordering

##### 2e: The features that have the largest absolute coefficient values contribute most to the linear regression model, showing that they have the strongest influence on sleep quality. The model has a low MSE value, which means the predictions are more accurate. Since the training and testing MSE values are similar, the model generalizes well and does not have significant overfitting.

In [131]:
#3a)
X_train = train_clean.drop(columns = [target]).to_numpy()
Y_train = train_clean[target].to_numpy()
X_test = test_clean.drop(columns = [target]).to_numpy()
Y_test = test_clean[target].to_numpy()

def fit_closed_form(X,Y):
    Xb = np.c_[np.ones((X.shape[0], 1)), X]
    theta = np.linalg.pinv(Xb.T @ Xb) @ (Xb.T @ Y)
    return theta

def predict_closed_form(X,theta):
    Xb = np.c_[np.ones((X.shape[0], 1)),X]
    return Xb @ theta

theta = fit_closed_form(X_train, Y_train)
intercept_cf = theta[0]
coefs_cf = theta[1:]

print("Closed-form intercept:", intercept_cf)
print("Closed-form coefficients:", coefs_cf)

Closed-form intercept: 4.614076015227511
Closed-form coefficients: [ 1.38404261e-02  6.63838357e-01 -5.94043432e-04 -3.22086472e-01
 -2.10175847e-02  9.15066625e-05]


In [132]:
#3b)
yhat_train_cf = predict_closed_form(X_train, theta)
yhat_test_cf = predict_closed_form(X_test, theta)
mse_train_cf = mean_squared_error(Y_train, yhat_train_cf)
r2_train_cf = r2_score(Y_train, yhat_train_cf)
mse_test_cf = mean_squared_error(Y_test, yhat_test_cf)
r2_test_cf = r2_score(Y_test, yhat_test_cf)


print("Closed Form Train MSE:", mse_train_cf)
print("Closed Form Train R2 :", r2_train_cf)
print("Closed Form Test MSE:", mse_test_cf)
print("Closed Form Test R2 :", r2_test_cf)

X_train_df = train_clean.drop(columns=[target])
X_test_df = test_clean.drop(columns=[target])
lr = LinearRegression()
lr.fit(X_train_df, train_clean[target])
yhat_train_sk = lr.predict(X_train_df)
yhat_test_sk = lr.predict(X_test_df)
mse_train_sk = mean_squared_error(train_clean[target], yhat_train_sk)
r2_train_sk = r2_score(train_clean[target], yhat_train_sk)
mse_test_sk = mean_squared_error(test_clean[target], yhat_test_sk)
r2_test_sk = r2_score(test_clean[target], yhat_test_sk)


print("sklearn Train MSE:", mse_train_sk)
print("sklearn Train R2 :", r2_train_sk)
print("sklearn Test MSE:", mse_test_sk)
print("sklearn Test R2 :", r2_test_sk)

Closed Form Train MSE: 0.13100264319632643
Closed Form Train R2 : 0.9064110852787419
Closed Form Test MSE: 0.12919025461879174
Closed Form Test R2 : 0.9127041171899675
sklearn Train MSE: 0.13100264319632646
sklearn Train R2 : 0.9064110852787418
sklearn Test MSE: 0.12919025461693848
sklearn Test R2 : 0.9127041171912198


##### 3b: The results are almost identical, which makes sense since both methods solve the same least squares optimization problem.

In [ ]:
#4a)
def poly_features(X,p):
    return np.column_stack([X**i for i in range(1, p+1)])

X_train = train_clean["Sleep_Duration"].to_numpy()
Y_train = train_clean[target].to_numpy()
X_test = test_clean["Sleep_Duration"].to_numpy()
Y_test = test_clean[target].to_numpy()
P = 2
X_train_poly = poly_features(X_train, P)
X_test_poly = poly_features(X_test, P)

theta_p = fit_closed_form(X_train_poly, Y_train)

yhat_train = predict_closed_form(X_train_poly, theta_p)
yhat_test = predict_closed_form(X_test_poly, theta_p)



##### 4a: I used polynomial regression by expanding the feature X into [X,X^2,...X^P] and fitting the model using the closed form least squares solutions. I made predictions by applying the learned parameters to the matrix

In [134]:
#4b)
results = []
for p in [1,2,3,5]:
    X_train_poly = poly_features(X_train, p)
    X_test_poly = poly_features(X_test, p)
    theta_p = fit_closed_form(X_train_poly, Y_train)
    yhat_train = predict_closed_form(X_train_poly, theta_p)
    yhat_test = predict_closed_form(X_test_poly, theta_p)

    results.append({
        "Polynomial Degree(p)": p,
        "Train MSE": mean_squared_error(Y_train, yhat_train),
        "Train R2": r2_score(Y_train, yhat_train),
        "Test MSE": mean_squared_error(Y_test, yhat_test),
        "Test R2": r2_score(Y_test, yhat_test)
    })
results_df = pd.DataFrame(results)
results_df

,Polynomial Degree(p),Train MSE,Train R2,Test MSE,Test R2
0,1,0.304066,0.782774,0.357803,0.758227
1,2,0.299796,0.785824,0.351232,0.762667
2,3,0.292250,0.791215,0.331388,0.776076
3,5,0.290183,0.792692,0.327831,0.778479


##### 4b: As the degree inceases, the training MSE decreases and the training R^2 increases, showing a better fit to the training data. Increase the degree to 5 improves the models ability to capture the relationship between sleep duration and sleep quality without a lot of overfitting.

##### 4c: As p increases, the training MSE decreases because the model gains more parameters and can fit the training data more closely. However, the testing MSE can eventually increase for larger p values because overly complex models may overfit the training data, and reducing generalization performance.

In [138]:
#5a)
X_train_df = train_clean.drop(columns=[target])
Y_train = train_clean[target].to_numpy()
X_test_df = test_clean.drop(columns=[target])
Y_test = test_clean[target].to_numpy()
X_train_raw = X_train_df.to_numpy(dtype=float)
X_test_raw = X_test_df.to_numpy(dtype=float)

def add_bias(X):
    return np.c_[np.ones((X.shape[0],1)),X]

def normalize_train_test(X_train, X_test):
    mu = X_train.mean(axis=0)
    sigma = X_train.std(axis=0)
    sigma[sigma == 0] = 1.0
    X_train_n = (X_train - mu)/sigma
    X_test_n = (X_test - mu)/sigma
    return X_train_n, X_test_n, mu, sigma

def gd_linear_regression(X,Y, alpha=0.1, iters=100):
    n, d = X.shape
    w = np.zeros(d)
    history = []

    for t in range(iters):
        yhat = X @ w
        err = yhat - Y
        grad = (2.0/n) * (X.T @ err)
        w = w - alpha * grad
        mse = np.mean(err**2)
        history.append(mse)
    return w, history

def eval_model(Xb, Y, w):
    yhat = Xb @ w
    return mean_squared_error(Y, yhat), r2_score(Y, yhat)

Xb_train_raw = add_bias(X_train_raw)
Xb_test_raw = add_bias(X_test_raw)

alpha_try = 0.1
w_raw, hist_raw = gd_linear_regression(Xb_train_raw, Y_train, alpha=1e-6, iters= 100)



X_train_n, X_test_n, mu, sigma = normalize_train_test(X_train_raw, X_test_raw)
Xb_train_n = add_bias(X_train_n)
Xb_test_n = add_bias(X_test_n)

w_norm, hist_norm = gd_linear_regression(Xb_train_n, Y_train, alpha=1e-6, iters = 100)

train_mse_norm, train_r2_norm = eval_model(Xb_train_n, Y_train, w_norm)
test_mse_norm, test_r2_norm = eval_model(Xb_test_n, Y_test, w_norm)

print("Theta without normalization:")
print(w_raw)
print("\nTheta with normalization:")
print(w_norm)



Theta without normalization:
[-4.92570559e+192 -2.09873388e+194 -3.51803253e+193 -3.14580975e+194
 -2.66199337e+193 -3.45255861e+194 -3.57417529e+196]

Theta with normalization:
[ 1.47385408e-03  1.08091190e-04  2.09301877e-04  3.87599077e-05
 -2.11778280e-04 -1.51590152e-04 -1.09932380e-05]


/var/folders/2t/h_cypqr94zx0kjmx1phwr9h40000gn/T/ipykernel_48895/943915138.py:30: RuntimeWarning: overflow encountered in square
  mse = np.mean(err**2)


##### 5a: I implemented gradient descent for linear regression. by adding a bias term, initializing weights to zero,a nd updating the formula to decrease the MSE. Without normalization graident decent can become unstable because the features have different scales. After normalizing each feture using the training mean and standard devation, training becomes stable.

In [136]:
#5b)
alphas = [0.01, 0.1, 0.5]
iters_list = [10, 50, 100]

rows = []

for alpha in alphas:
    for iters in iters_list:
        w, _ = gd_linear_regression(
            Xb_train_n, Y_train, alpha=alpha, iters=iters
        )
        train_mse, train_r2 = eval_model(Xb_train_n, Y_train, w)
        test_mse, test_r2 = eval_model(Xb_test_n, Y_test, w)
        rows.append({
            "alpha": alpha,
            "iterations": iters,
            "Train MSE": train_mse,
            "Train R2": train_r2,
            "Test MSE": test_mse,
            "Test R2": test_r2
        })
results = pd.DataFrame(rows)
results

,alpha,iterations,Train MSE,Train R2,Test MSE,Test R2
0,0.01,10,3.686246e+01,-2.533472e+01,3.477731e+01,-2.249957e+01
1,0.01,50,7.357906e+00,-4.256523e+00,7.201585e+00,-3.866224e+00
2,0.01,100,1.095132e+00,2.176324e-01,1.072739e+00,2.751334e-01
3,0.10,10,7.656346e-01,4.530270e-01,7.475378e-01,4.948770e-01
4,0.10,50,1.325647e-01,9.052952e-01,1.276342e-01,9.137556e-01
5,0.10,100,1.312959e-01,9.062016e-01,1.282910e-01,9.133117e-01
6,0.50,10,8.683347e+03,-6.202425e+03,9.328417e+03,-6.302358e+03
7,0.50,50,2.660853e+19,-1.900926e+19,2.858224e+19,-1.931347e+19
8,0.50,100,6.066612e+38,-4.334017e+38,6.516609e+38,-4.403375e+38


##### 5c: With a small learning rate, the gradient descent converges slowly. Early iterations have high MSE and negative R^2, and reasonable performance is reached only have a lot of interations. A moderate learning rate converges quickly and stably, which means the MSE is low and the R^2 is high by about 50 iterations. A large learning rate causes divergence, with a large MSE and a negative R^2. Overall normalization and an appropriate learning rate allow gradient descent to converge to the optimal solution efficiently. 